In [43]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from datasets import Dataset

# Load Dataset

In [20]:
data_path = "../data/jutsus.jsonl"
df = pd.read_json(data_path, lines=True)
df.head()

,jutsu_name,jutsu_type,jutsu_description
0,Adamantine Sealing Chains: Spiral Formation,"Hiden, Ninjutsu, Fūinjutsu, Barrier Ninjutsu, ...",Kushina uses her chains to form a barrier whil...
1,Adamantine Power: Acala,"Kekkei Genkai, Ninjutsu, Taijutsu",Hashirama kicks the opponent away and raises s...
2,Adamantine Prison Wall,"Ninjutsu, Clone Techniques, Bukijutsu","After using Transformation: Adamantine Staff, ..."
3,Adamantine Seal: Monkey Yang Suppression,"Ninjutsu, Fūinjutsu, Cooperation Ninjutsu","After placing fūinjutsu tags in an area, the u..."
4,Acrobat,"Taijutsu, Kenjutsu","The Acrobat (荒繰鷺伐刀, Akurobatto) is a kenjutsu ..."


In [21]:
def simplify_jutsu(jutsu):
    if "Genjutsu" in jutsu:
        return "Genjutsu"
    if "Ninjutsu" in jutsu:
        return "Ninjutsu"
    if "Taijutsu" in jutsu:
        return "Taijutsu"

In [22]:
df['jutsu_type_simplified'] = df['jutsu_type'].apply(simplify_jutsu)

In [23]:
df.head()

,jutsu_name,jutsu_type,jutsu_description,jutsu_type_simplified
0,Adamantine Sealing Chains: Spiral Formation,"Hiden, Ninjutsu, Fūinjutsu, Barrier Ninjutsu, ...",Kushina uses her chains to form a barrier whil...,Ninjutsu
1,Adamantine Power: Acala,"Kekkei Genkai, Ninjutsu, Taijutsu",Hashirama kicks the opponent away and raises s...,Ninjutsu
2,Adamantine Prison Wall,"Ninjutsu, Clone Techniques, Bukijutsu","After using Transformation: Adamantine Staff, ...",Ninjutsu
3,Adamantine Seal: Monkey Yang Suppression,"Ninjutsu, Fūinjutsu, Cooperation Ninjutsu","After placing fūinjutsu tags in an area, the u...",Ninjutsu
4,Acrobat,"Taijutsu, Kenjutsu","The Acrobat (荒繰鷺伐刀, Akurobatto) is a kenjutsu ...",Taijutsu


In [24]:
df['jutsu_type_simplified'].value_counts()

jutsu_type_simplified
Ninjutsu    2255
Taijutsu     397
Genjutsu     101
Name: count, dtype: int64

In [27]:
df['text'] = df['jutsu_name'] + ". " + df['jutsu_description']
df['jutsus'] = df['jutsu_type_simplified']
df = df[['text', 'jutsus']]
df = df.dropna()

In [26]:
df.head()

,jutsu_name,jutsu_type,jutsu_description,jutsu_type_simplified,text,jutus
0,Adamantine Sealing Chains: Spiral Formation,"Hiden, Ninjutsu, Fūinjutsu, Barrier Ninjutsu, ...",Kushina uses her chains to form a barrier whil...,Ninjutsu,Adamantine Sealing Chains: Spiral Formation. K...,Ninjutsu
1,Adamantine Power: Acala,"Kekkei Genkai, Ninjutsu, Taijutsu",Hashirama kicks the opponent away and raises s...,Ninjutsu,Adamantine Power: Acala. Hashirama kicks the o...,Ninjutsu
2,Adamantine Prison Wall,"Ninjutsu, Clone Techniques, Bukijutsu","After using Transformation: Adamantine Staff, ...",Ninjutsu,Adamantine Prison Wall. After using Transforma...,Ninjutsu
3,Adamantine Seal: Monkey Yang Suppression,"Ninjutsu, Fūinjutsu, Cooperation Ninjutsu","After placing fūinjutsu tags in an area, the u...",Ninjutsu,Adamantine Seal: Monkey Yang Suppression. Afte...,Ninjutsu
4,Acrobat,"Taijutsu, Kenjutsu","The Acrobat (荒繰鷺伐刀, Akurobatto) is a kenjutsu ...",Taijutsu,"Acrobat. The Acrobat (荒繰鷺伐刀, Akurobatto) is a ...",Taijutsu


In [10]:
from bs4 import BeautifulSoup
class Cleaner():
    def __init__(self):
        pass 
    
    def put_line_breaks(self, text):
        return text.replace("<\p>", "<\p>\n")
    
    def remove_html_tags(self, text):
        clean_text = BeautifulSoup(text, "lxml").text
        return clean_text

    def clean(self, text):
        text = self.put_line_breaks(text)
        text = self.remove_html_tags(text)
        text = text.strip()
        return text

In [18]:
text_column_name = 'text'
label_column_name = "jutsus"

In [28]:
# Clean Text
cleaner = Cleaner()
df['text_cleaned'] = df[text_column_name].apply(cleaner.clean)

/tmp/ipykernel_25661/3655626712.py:10: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  clean_text = BeautifulSoup(text, "lxml").text


In [29]:
df.head(2)

,text,jutsus,text_cleaned
0,Adamantine Sealing Chains: Spiral Formation. K...,Ninjutsu,Adamantine Sealing Chains: Spiral Formation. K...
1,Adamantine Power: Acala. Hashirama kicks the o...,Ninjutsu,Adamantine Power: Acala. Hashirama kicks the o...


In [30]:
# Encode Labels 
le = preprocessing.LabelEncoder()
le.fit(df[label_column_name].tolist())

LabelEncoder()

In [31]:
label_dict = {index:label_name for index, label_name in enumerate(le.__dict__['classes_'].tolist())}
label_dict

{0: 'Genjutsu', 1: 'Ninjutsu', 2: 'Taijutsu'}

In [32]:
df['label'] = le.transform(df[label_column_name].tolist())

In [33]:
df.head()

,text,jutsus,text_cleaned,label
0,Adamantine Sealing Chains: Spiral Formation. K...,Ninjutsu,Adamantine Sealing Chains: Spiral Formation. K...,1
1,Adamantine Power: Acala. Hashirama kicks the o...,Ninjutsu,Adamantine Power: Acala. Hashirama kicks the o...,1
2,Adamantine Prison Wall. After using Transforma...,Ninjutsu,Adamantine Prison Wall. After using Transforma...,1
3,Adamantine Seal: Monkey Yang Suppression. Afte...,Ninjutsu,Adamantine Seal: Monkey Yang Suppression. Afte...,1
4,"Acrobat. The Acrobat (荒繰鷺伐刀, Akurobatto) is a ...",Taijutsu,"Acrobat. The Acrobat (荒繰鷺伐刀, Akurobatto) is a ...",2


In [35]:
test_size = 0.2
df_train, df_test = train_test_split(df, 
                                     test_size=test_size, 
                                     stratify=df['label'],)

In [37]:
df_train['jutsus'].value_counts()

jutsus
Ninjutsu    1804
Taijutsu     317
Genjutsu      81
Name: count, dtype: int64

In [38]:
model_name = "distilbert/distilbert-base-uncased"

In [40]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

/home/abdullah/python_virtual_env/gradio_env/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [41]:
def preprocess_function(tokenizer,examples):
    return tokenizer(examples['text_cleaned'],truncation=True)

In [44]:
# Conver Pandas to a hugging face dataset
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)

# tokenize the dataset
tokenized_train = train_dataset.map(lambda examples: preprocess_function(tokenizer, examples),
                                    batched=True)
tokenized_test = test_dataset.map(lambda examples: preprocess_function(tokenizer, examples),
                                    batched=True)

Map:   0%|          | 0/2202 [00:00<?, ? examples/s]

Map:   0%|          | 0/551 [00:00<?, ? examples/s]